<a href="https://colab.research.google.com/github/Vinod-IE/Budget-chatbot/blob/main/Multi_Rag_vinod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install "openai>=1" "langchain>=0.0.331rc2" "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib chromadb tiktoken watermark

In [42]:
!sudo apt-get install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [43]:
!watermark -a "Vinod" -vmp langchain,unstructured,openai

/bin/bash: line 1: watermark: command not found


In [44]:
path = "/content/"

In [45]:
from unstructured.partition.pdf import partition_pdf
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import openai
import os



In [46]:
# Extract images, tables, and chunk text
path = "/content/"
raw_pdf_elements = partition_pdf(
    filename=path + "FY2025_m1.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)

# Categorize by type
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))




In [49]:
# Define the prompt for question-answering
qa_prompt_text = """You are an assistant that retrieves information from tables. \
Given a table and a question, find the cell in the table that contains the answer to the question. \
If the question is about a specific row and column, return the value from that cell. \
Table: {table}
Question: {question}"""

qa_prompt = ChatPromptTemplate.from_template(qa_prompt_text)

# Define the model
model = ChatOpenAI(temperature=0, model="gpt-4")

# Function to retrieve the answer from a table
def retrieve_answer_from_table(table, question):
    # Combine the table and question into the prompt
    prompt_text = qa_prompt.format(table=table, question=question)

    # Generate the response
    response = model(prompt_text)

    # Debug: print the type and content of the response
    print(f"Response type: {type(response)}")
    print(f"Response content: {response}")

    # Ensure the response is a string before parsing
    if not isinstance(response, str):
        raise TypeError(f"Expected response to be of type str, but got {type(response)}")

    # Parse the response to extract the answer
    parser = StrOutputParser()
    answer = parser.parse(response)

    return answer

# Example question
question = "What is the Grand Total Direct - Military Personnel in FY 2023 Actuals?"

# Test the function with the first table extracted
if tables:
    table = tables[0]
    try:
        answer = retrieve_answer_from_table(table, question)
        print("Answer:", answer)
    except Exception as e:
        print(f"Error: {e}")
else:
    print("No tables found in the PDF.")

Error: Got unknown type H
